In [ ]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split

from scipy import stats
from dython.nominal import associations

import seaborn as sns
import matplotlib.pyplot  as plt

In [ ]:
train_name='train_incident'
validation_name='validation_incident'
test_name='test_incident'

projectId='pongthorn'
dataset_id='DemoSMartDW'

In [7]:

dw_table_id = f"{projectId}.{dataset_id}.incident"

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
val_tabel_id=f"{projectId}.{dataset_id}.{validation_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"

file_name="ML_Incident.csv"

In [8]:
label='severity_id'
labelName='severity_name'

threshold_x_sd = 3  # 2.5/3/3.5  3SD

removeCols=['id','updated_at','imported_at']

dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

numbericCols=['count_detail','open_to_close_hour','open_to_response_hour','response_to_resolved_hour']

cateCols=['sla','product_type','brand','service_type','incident_type']


# Load  Data from BigQuery

In [26]:
def load_data_bq(sql:str):

 query_result=client.query(sql)
 df_all=query_result.to_dataframe()
 return df_all


In [28]:
imported_to='2023-04-04' 

sql_all=f"""
SELECT  id,severity_id,severity_name
,count_detail
,sla,product_type,brand,service_type,incident_type
,open_datetime,  close_datetime, response_datetime,resolved_datetime
,updated_at,imported_at

FROM `{dw_table_id}`    

WHERE imported_at< '{imported_to}'

order  by imported_at
"""

print(sql_all)


SELECT  id,severity_id,severity_name
,count_detail
,sla,product_type,brand,service_type,incident_type
,open_datetime,  close_datetime, response_datetime,resolved_datetime
,updated_at,imported_at

FROM `pongthorn.DemoSMartDW.incident`    

WHERE imported_at< '2023-04-04'

order  by imported_at



In [29]:
print("Load data from Bigquery")
df_all=load_data_bq(sql_all)
if len(df_all)==0:
 print("No records from bigquery")  
 quit()
else:
 print(df_all.shape)   

Load data from Bigquery
(2418, 15)


In [30]:
df_all=df_all.drop_duplicates(subset=['id'],keep='last')
no_original_rows=len(df_all)

df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2418 entries, 0 to 2417
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2418 non-null   Int64         
 1   severity_id        2418 non-null   Int64         
 2   severity_name      2418 non-null   object        
 3   count_detail       2418 non-null   Int64         
 4   sla                2418 non-null   object        
 5   product_type       2418 non-null   object        
 6   brand              2418 non-null   object        
 7   service_type       2418 non-null   object        
 8   incident_type      2418 non-null   object        
 9   open_datetime      2418 non-null   datetime64[ns]
 10  close_datetime     2418 non-null   datetime64[ns]
 11  response_datetime  2418 non-null   datetime64[ns]
 12  resolved_datetime  2418 non-null   datetime64[ns]
 13  updated_at         2418 non-null   datetime64[ns]
 14  imported

In [31]:
start_end_list=[ ['open_datetime','close_datetime'],['open_datetime','response_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )

df_all[listDiffHourCols].tail(5)

,open_to_close_hour,open_to_response_hour,response_to_resolved_hour
2413,9.466667,0.016667,9.450000
2414,96.450000,1.116667,77.650000
2415,0.883333,0.166667,0.716667
2416,4.666667,0.183333,4.483333
2417,5.483333,0.000000,5.483333


In [32]:
df_all[listDiffHourCols].describe(percentiles=[.95,.75,.50,.25,.05])

,open_to_close_hour,open_to_response_hour,response_to_resolved_hour
count,2418.000000,2418.000000,2418.000000
mean,174.198229,3.967894,133.719279
std,739.248077,39.531696,681.510174
min,0.000000,0.000000,0.000000
5%,0.966667,0.000000,0.166667
25%,4.066667,0.050000,2.670833
50%,8.891667,0.166667,6.666667
75%,48.000000,0.500000,26.000000
95%,810.216667,7.858333,503.682500
max,13177.250000,1152.133333,13176.250000


In [33]:
# https://www.geeksforgeeks.org/z-score-for-outlier-detection-python/
xScoreDiffHourCols=[]
for col in listDiffHourCols:
  z_col=f"zscore_{col}"   
  df_all[z_col] = np.abs(stats.zscore(df_all[col]))   
  xScoreDiffHourCols.append(z_col)  
#remove oulier

for col in  xScoreDiffHourCols:
  df_all=df_all.query(f"{col}<@threshold_x_sd")


no_rows_after_removing_outlier=len(df_all)
pct_row_decrease=round( (no_original_rows-no_rows_after_removing_outlier)/no_original_rows*100 ,0)

print(f"% remove data {pct_row_decrease}")

print(df_all.info())

% remove data 2.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2367 entries, 0 to 2417
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype          
---  ------                            --------------  -----          
 0   id                                2367 non-null   Int64          
 1   severity_id                       2367 non-null   Int64          
 2   severity_name                     2367 non-null   object         
 3   count_detail                      2367 non-null   Int64          
 4   sla                               2367 non-null   object         
 5   product_type                      2367 non-null   object         
 6   brand                             2367 non-null   object         
 7   service_type                      2367 non-null   object         
 8   incident_type                     2367 non-null   object         
 9   open_datetime                     2367 non-null   datetime64[ns] 
 10  close_datetime    

In [34]:
allRows=df_all.shape[0]
for col in numbericCols:
    zeroRows=len(df_all.query(f"{col}==0"))
    pctZeroHour=round(zeroRows/allRows*100,1)
    print(f"No. 0-hour row on {col} = {zeroRows}({pctZeroHour} %)")
    


No. 0-hour row on count_detail = 116(4.9 %)
No. 0-hour row on open_to_close_hour = 1(0.0 %)
No. 0-hour row on open_to_response_hour = 454(19.2 %)
No. 0-hour row on response_to_resolved_hour = 99(4.2 %)


In [35]:
def explore_ranges_numberic_val(col,rangeList):
    print(col)
    rangeList.sort()
    return pd.cut(df_all[col],rangeList, right=True).value_counts()

range1= [0,24, 168, 360, 720,math.floor(df_all['open_to_close_hour'].max())]
print(explore_ranges_numberic_val('open_to_close_hour',range1))
print("=======================================================")

range2= [0,24, 168, 360, 720,math.floor(df_all['response_to_resolved_hour'].max())]
print(explore_ranges_numberic_val('response_to_resolved_hour',range2))
print("=======================================================")



open_to_close_hour
(0, 24]        1602
(24, 168]       484
(168, 360]      121
(720, 2349]      94
(360, 720]       64
Name: open_to_close_hour, dtype: int64
response_to_resolved_hour
(0, 24]        1673
(24, 168]       390
(168, 360]       96
(720, 2016]      62
(360, 720]       46
Name: response_to_resolved_hour, dtype: int64


In [36]:
range4= [0,2, 4,math.floor(df_all['count_detail'].max())]
print(explore_ranges_numberic_val('count_detail',range4))
print("=======================================================")

count_detail
(0, 2]     1787
(2, 4]      281
(4, 36]     183
Name: count_detail, dtype: int64


In [37]:
df_all=df_all.query('response_to_resolved_hour!=0')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268 entries, 0 to 2417
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype          
---  ------                            --------------  -----          
 0   id                                2268 non-null   Int64          
 1   severity_id                       2268 non-null   Int64          
 2   severity_name                     2268 non-null   object         
 3   count_detail                      2268 non-null   Int64          
 4   sla                               2268 non-null   object         
 5   product_type                      2268 non-null   object         
 6   brand                             2268 non-null   object         
 7   service_type                      2268 non-null   object         
 8   incident_type                     2268 non-null   object         
 9   open_datetime                     2268 non-null   datetime64[ns] 
 10  close_datetime                    22

# Drop unselected columns

In [39]:
numColsRemove=['open_to_response_hour','count_detail'] 
numbericCols=[x for x in numbericCols if x not in numColsRemove]   
numbericCols

removeCols=removeCols+dateCols+xScoreDiffHourCols+listDiffDateDeltaCols+numColsRemove
df_all=df_all.drop(columns=removeCols)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268 entries, 0 to 2417
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_id                2268 non-null   Int64  
 1   severity_name              2268 non-null   object 
 2   sla                        2268 non-null   object 
 3   product_type               2268 non-null   object 
 4   brand                      2268 non-null   object 
 5   service_type               2268 non-null   object 
 6   incident_type              2268 non-null   object 
 7   open_to_close_hour         2268 non-null   float64
 8   response_to_resolved_hour  2268 non-null   float64
dtypes: Int64(1), float64(2), object(6)
memory usage: 179.4+ KB


# Visualize  Data